# Few-shot Learning

- **정의**: 이미 학습된 모델에 대해 새로운 예시 상황을 몇 가지 예제로 추가 학습시키는 방식입니다. 모델은 이 예시들을 통해 문제 해결 방식이나 답변 형식을 파악하여 유사한 질문에 대한 추론 성능을 향상시킵니다. **Zero-shot learning**과 달리 모델이 일부 예시를 접한 후 이를 바탕으로 예측을 수행합니다.
  
- **Selector**: `selector`를 사용하여 Few-shot learning에서 사용할 **예시를 선별**할 수 있습니다. `selector`는 예시 길이, 내용 등 다양한 기준에 따라 예시를 선택하는 역할을 하며, 커스텀으로 설정하거나 기본 제공되는 길이 기반 `selector`를 사용할 수 있습니다. 이를 통해 모델이 보다 적합한 예시를 학습할 수 있습니다.


In [2]:
# few shot learning 을 위한 예시이다. 
# few show learning은 zero shot learning을 대체하는 것으로 조금이라도 본 예시가 있으면 모델이 이를 잘 훈련해서 
# 추론을 해낼 수 있게 되는 것이다. zero shot learning은 한 번도 본 적 없는 상황에 맞닥뜨려도 추론을 해내는 것을 의미한다. 
examples = [
{
"question": "What do you know about France?",
"answer": """
Here is what I know:
Capital: Paris
Language: French
Food: Wine and Cheese
Currency: Euro
""",
},
{
"question": "What do you know about Italy?",
"answer": """
I know this:
Capital: Rome
Language: Italian
Food: Pizza and Pasta
Currency: Euro
""",
},
{
"question": "What do you know about Greece?",
"answer": """
I know this:
Capital: Athens
Language: Greek
Food: Souvlaki and Feta Cheese
Currency: Euro
""",
},
]



In [6]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate, FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.example_selector.base import BaseExampleSelector
from langchain.prompts.example_selector import LengthBasedExampleSelector

# 아래 구조가 달라지면 안 된다. add_example과 select_examples는 이름이 바뀌면 안 되니 주의

class RandomExampleSelector(BaseExampleSelector):
    
    def __init__(self, examples):
        self.examples = examples
        
    def add_example(self, example):
        self.examples.append(example)
    
    def select_examples(self, input_variables):
        from random import choice
        return [choice(self.examples)]

# streaming = True / 과 다음의 callbacks 옵션을 주면 출력값을 순차적으로 확인할 수 있다.
chat = ChatOpenAI(
    temperature=0.1, 
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

# 위의 예시와 동일한 key_value 구조로 들어와야 한다. 
example_template = """
    Human:{question},
    AI:{answer}
"""
example_prompt = PromptTemplate.from_template(example_template)

selector = True

if selector :
    # 아래 selector 2개 중 하나 선택할 것
    example_selector = RandomExampleSelector(
        examples=examples
    )
    # example_selector = LengthBasedExampleSelector(
    #     examples=examples,
    #     example_prompt = example_prompt,
    #     max_length=10,
    # )
    prompt = FewShotPromptTemplate(
        example_prompt=example_prompt,
        example_selector = example_selector,
        suffix="Human: What do you know about {country}?",
        input_variables=["country"]
    )
else:
    prompt = FewShotPromptTemplate(
        example_prompt=example_prompt,
        examples=examples,
        suffix="Human: What do you know about {country}?",
        input_variables=["country"]
    )
chain = prompt | chat
chain.invoke({
    "country":"South Korea"
})


AI:
I know this:
Capital: Seoul
Language: Korean
Food: Kimchi, Bibimbap, Bulgogi
Currency: South Korean Won
Famous for: K-pop, technology, and beautiful landscapes

AIMessageChunk(content='AI:\nI know this:\nCapital: Seoul\nLanguage: Korean\nFood: Kimchi, Bibimbap, Bulgogi\nCurrency: South Korean Won\nFamous for: K-pop, technology, and beautiful landscapes')

In [12]:
examples = [
{
"country": "France",
"answer": """
Here is what I know:
Capital: Paris
Language: French
Food: Wine and Cheese
Currency: Euro
etc: 
""",
},
{
"country": "Italy",
"answer": """
I know this:
Capital: Rome
Language: Italian
Food: Pizza and Pasta
Currency: Euro
etc: 
""",
},
{
"country": "Greece",
"answer": """
I know this:
Capital: Athens
Language: Greek
Food: Souvlaki and Feta Cheese
Currency: Euro
etc: 
""",
},
]

In [15]:
chat = ChatOpenAI(
    temperature=0.1, 
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

example_prompt = ChatPromptTemplate.from_messages([
    ("human", "What do you know about {country}?"),
    ("ai", "{answer}")    
])

selector = False

if selector :
    # 2개의 selector 중에서 하나 선택
    example_selector = RandomExampleSelector(
        examples=examples
    )
    # example_selector = LengthBasedExampleSelector(
    #     examples=examples,
    #     example_prompt = example_prompt,
    #     max_length=10,
    # )
    example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    example_selector = example_selector
    )
else:    
    example_prompt = FewShotChatMessagePromptTemplate(
        example_prompt=example_prompt,
        examples=examples,
    )

final_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a geograpy expert."),
    example_prompt,
    ("human","What do you know about {country}?")
])

chain = final_prompt | chat

chain.invoke({
    "country":"South Korea"
})


I know this:
Capital: Seoul
Language: Korean
Food: Kimchi and Bibimbap
Currency: South Korean Won
etc: 


AIMessageChunk(content='\nI know this:\nCapital: Seoul\nLanguage: Korean\nFood: Kimchi and Bibimbap\nCurrency: South Korean Won\netc: \n')